In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.colors import ListedColormap
import matplotlib as mpl
# pd.set_option('display.max_rows', None)
from datetime import datetime,timedelta
import seaborn as sns
from scipy import stats
import folium
import geopandas as gpd
print(sys.version)
print(np.__version__)
print(pd.__version__)

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
1.17.2
0.25.1


---
# Make recommendations to increase ridership.

### Recommendations
1. Incentivize users to rider during November and December.  Through my analysis, it became apparent that the worst months of the year were November and December.

2. Increase ridership during the weekends.  The best year for weekend trips was 2014.  Despite the total duration of weekday tips increasing in 2015, when compared to 2014, it ended up finishing with fewer total hours of trips.  The nosedive in weekend trips significantly contributed to 2015 logging fewer hours than 2014.  If the total weekend hours per day were more in line with the total weekday hours per day, a substantial increase in ridership could be realized. 

3. Add more bikes and docks to the top ten most popular stations.  The top ten most popular stations (in terms of total trip duration) have a 15% occurrence rate of zero to two bikes or zero to two docking stations available during the hours of 5 AM to 8 PM (the busiest time of the day).  Also, the top three bike stations supply is low upwards of 20% during this time frame.

For further explanation please continue reading.

### Setup

Before answering this question, I will save a view that contains trips that are eight hours or less--99.8% of trips are eight hours or less.  Therefore trips more than eight hours are outliers and will skew the results I intend to deliver.  However, there is one caveat, with the top 0.2% of data trimmed relating the bikeshare_trip table to the other tables will be slightly impacted.  This misrepresentation will be insignificant towards offer recommendations to increase ridership. 

The view I created is: studied-bounty-276502:project_01.trips_lesseq_8hr. 

*SQL Query*
```
SELECT *
FROM bigquery-public-data.san_francisco.bikeshare_trips 
WHERE duration_sec <= 28800
```

In [2]:
! bq query --use_legacy_sql=FALSE 'SELECT EXTRACT(year FROM start_date) AS date_year, \
round(sum(duration_sec/60),0) as total_duration_min \
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr` \
GROUP BY date_year \
ORDER BY date_year DESC'

/bin/sh: bq: command not found


In [3]:
! bq query --use_legacy_sql=FALSE 'SELECT EXTRACT(year FROM start_date) AS date_year, \
round(sum(duration_sec/3600),0) as total_duration_hr \
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr` \
GROUP BY date_year \
ORDER BY date_year DESC'

/bin/sh: bq: command not found


## Start of magic commads to make dataframes.

#### Trips by Duration and Origin/Destintaion

In [4]:
%%bigquery dur_trips_orgdest_2016
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
end_station_id,
end_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/2692370)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3,4,5
ORDER BY total_trip_time DESC)
WHERE date_year = 2016

UsageError: Cell magic `%%bigquery` not found.


In [ ]:
%%bigquery dur_trips_orgdest_2015
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
end_station_id,
end_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/4578562)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3,4,5
ORDER BY total_trip_time DESC)
WHERE date_year = 2015

In [ ]:
%%bigquery dur_trips_orgdest_2014
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
end_station_id,
end_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/4673800)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3,4,5
ORDER BY total_trip_time DESC)
WHERE date_year = 2014

#### Trips by Number of Trips and Origin/Destintaion

In [ ]:
%%bigquery pop_trips_orgdest_2016
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
end_station_id,
end_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/2692370)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3,4,5
ORDER BY num_trips DESC)
WHERE date_year = 2016

In [ ]:
%%bigquery pop_trips_orgdest_2015
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
end_station_id,
end_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/4578562)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3,4,5
ORDER BY num_trips DESC)
WHERE date_year = 2015

In [ ]:
%%bigquery pop_trips_orgdest_2014
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
end_station_id,
end_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/4673800)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3,4,5
ORDER BY num_trips DESC)
WHERE date_year = 2014

#### Trips by Duration and Origin

In [ ]:
%%bigquery dur_trips_org_2016
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/2692370)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3
ORDER BY total_trip_time DESC)
WHERE date_year = 2016

In [ ]:
%%bigquery dur_trips_org_2015
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/4578562)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3
ORDER BY total_trip_time DESC)
WHERE date_year = 2015

In [ ]:
%%bigquery dur_trips_org_2014
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/4673800)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3
ORDER BY total_trip_time DESC)
WHERE date_year = 2014

In [ ]:
%%bigquery pop_trips_org_2016
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/2692370)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3
ORDER BY num_trips DESC)
WHERE date_year = 2016

In [ ]:
%%bigquery pop_trips_org_2015
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/4578562)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3
ORDER BY num_trips DESC)
WHERE date_year = 2015

In [ ]:
%%bigquery pop_trips_org_2014
SELECT *
FROM(
SELECT extract(year from start_date) AS date_year,
start_station_id,
start_station_name,
count(distinct trip_id) AS num_trips,
round(sum(duration_sec)/60,0) AS total_trip_time,
round(sum((((duration_sec)/60)/4673800)*100),2) AS percent_trip_time,
round(avg(duration_sec/60),0) AS avg_trip_time,
round(min(duration_sec/60),1) AS min_trip_time,
round(max(duration_sec/60),1) AS max_trip_time,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY 1,2,3
ORDER BY num_trips DESC)
WHERE date_year = 2014

## Plot Stations

In [ ]:
%%bigquery bike_stations
SELECT * FROM `bigquery-public-data.san_francisco.bikeshare_stations`

In [ ]:
g = gpd.GeoDataFrame(bike_stations, geometry=gpd.points_from_xy(bike_stations.longitude, bike_stations.latitude))
m = folium.Map(location=[g.latitude.mean(), g.longitude.mean()])
points = folium.features.GeoJson(g[['geometry']].to_json())
m.add_child(points)

## Start of Analysis

### 1.0 Trips by Number of Trips and Origin/Destination

#### 1.1.1 - 2016 Trips by Number of Trips and Origin/Destination

In [ ]:
pop_trips_orgdest_2016.head(10)

##### 2016 Top Ten Trips by Number of Trips and Origin/Destination
* Total trip duration percent = 7.1%

In [ ]:
round(pop_trips_orgdest_2016.percent_trip_time.head(10).sum(),2)

#### 1.1.2 - 2015 Trips by Number of Trips and Origin/Destination

In [ ]:
pop_trips_orgdest_2015.head(10)

##### 2015 Top Ten Trips by Number of Trips and Origin/Destination
* Total trip duration percent = 5.84%

In [ ]:
round(pop_trips_orgdest_2015.percent_trip_time.head(10).sum(),2)

#### 1.1.3 - 2014 Trips by Number of Trips and Origin/Destination

In [ ]:
pop_trips_orgdest_2014.head(10)

##### 2014 Top Ten Trips by Number of Trips and Origin/Destination
* Total trip duration percent = 5.0%

In [ ]:
round(pop_trips_orgdest_2014.percent_trip_time.head(10).sum())

### 1.2 Summary
After grouping trips by start station and end station and then ordering by the number of trips, the data reveals the top ten trips make up less than 10% of total duration.    

This is a decent start for deriving recommendations; however, I will move on to further analysis.  I am looking to capture more of the population before making recommendations.

### 2.0 Trips by Total Duration and Origin/Destination

#### 2.1.1 - 2016 Trips by Duration and Origin/Destination

In [ ]:
dur_trips_orgdest_2016.head(10)

##### 2016 Top Ten Trips by Duration and Origin/Destination
* Total trip duration percent = 9.0%

In [ ]:
round(dur_trips_orgdest_2016.percent_trip_time.head(10).sum())

#### 2.1.2 - 2015 Trips by Duration and Origin/Destination

In [ ]:
dur_trips_orgdest_2015.head(10)

##### 2015 Top Ten Trips by Duration and Origin/Destination
* Total trip duration percent = 8.0%

In [ ]:
round(dur_trips_orgdest_2015.percent_trip_time.head(10).sum())

#### 2.1.3 - 2014 Trips by Duration and Origin/Destination

In [ ]:
dur_trips_orgdest_2014.head(10)

##### 2014 Top Ten Trips by Duration and Origin/Destination
* Total trip duration percent = 9.0%

In [ ]:
round(dur_trips_orgdest_2014.percent_trip_time.head(10).sum())

### 2.2 Summary
After grouping trips by start station and end station and then ordering by duration, the data reveals the top ten trips (sorted by duration) make up less than 10% of total duration.  It is interesting to note the order of trip magnitude differs from the duration table to the number of trips table.  When ordering by duration, the number of trips was in the hundreds vs. the number of trips is in the thousands, when ordering by the number of trips.This is a decent start for deriving recommendations; However, I will move on to further analysis.  I am looking to capture more of the population before making recommendations.

The trips grouped by start station and end station are limiting the percent of the total duration.  Next, I will sort by the orgin only.

### 3.0 Trips by Duration and Number of Trips

#### 3.1.1 - 2016 Trips by Duration and Number of Trips

##### 3.1.1.1 - 2016 Trips by Duration

In [ ]:
dur_trips_org_2016.head(10)

##### 3.1.1.2 - 2016 Trips by Number of Trips

In [ ]:
pop_trips_org_2016.head(10)

##### 2016 Top Ten Trip Origin Summary
* Total trip duration percent = 46%
* Total number of trips percent = 46% 
* Total number of trips = 210,237

In [ ]:
round(dur_trips_org_2016.percent_trip_time.head(10).sum()), \
round((dur_trips_org_2016.num_trips.head(10).sum()/dur_trips_org_2016.num_trips.sum())*100), \
dur_trips_org_2016.num_trips.sum()

In [ ]:
round(pop_trips_org_2016.percent_trip_time.head(10).sum()), \
round((pop_trips_org_2016.num_trips.head(10).sum()/pop_trips_org_2016.num_trips.sum())*100), \
pop_trips_org_2016.num_trips.sum()

#### 3.1.2 - 2015 Trips by Duration and Number of Trips

##### 3.1.2.1 - 2015 Trips by Duration

In [ ]:
dur_trips_org_2015.head(10)

##### 3.1.2.1 - 2015 Trips by Number of Trips

In [ ]:
pop_trips_org_2015.head(10)

##### 2015 Top Ten Trip Origin Summary
* Total trip duration percent = 45%
* Total number of trips percent = 44% 
* Total number of trips = 345,634

In [ ]:
round(dur_trips_org_2015.percent_trip_time.head(10).sum()), \
round((dur_trips_org_2015.num_trips.head(10).sum()/dur_trips_org_2015.num_trips.sum())*100), \
dur_trips_org_2015.num_trips.sum()

In [ ]:
round(pop_trips_org_2015.percent_trip_time.head(10).sum()), \
round((pop_trips_org_2015.num_trips.head(10).sum()/pop_trips_org_2015.num_trips.sum())*100), \
pop_trips_org_2015.num_trips.sum()

#### 3.1.3 - 2014 Trips by Duration and Number of Trips

##### 3.1.3.1 - 2013 Trips by Duration

In [ ]:
dur_trips_org_2014.head(10)

##### 3.1.3.2 - 2013 Trips by Number of Trips

In [ ]:
pop_trips_org_2014.head(10)

##### 2014 Top Ten Trip Origin Summary
* Total trip duration percent = 43%
* Total number of trips percent = 41% 
* Total number of trips = 325,322

In [ ]:
round(dur_trips_org_2014.percent_trip_time.head(10).sum()), \
round((dur_trips_org_2014.num_trips.head(10).sum()/dur_trips_org_2014.num_trips.sum())*100), \
dur_trips_org_2014.num_trips.sum()

In [ ]:
round(pop_trips_org_2014.percent_trip_time.head(10).sum()), \
round((pop_trips_org_2014.num_trips.head(10).sum()/pop_trips_org_2014.num_trips.sum())*100), \
pop_trips_org_2014.num_trips.sum()

### 3.2 Summary
Grouping by the starting station captured 46% of the total trip duration and the total number of trips.  This is significantly more compared to grouping by the start and end station--10% of the total duration.  After capturing nearly 50% of the trips, any recommendations are sure to make more of an impact.

Further analysis will incorporate the starting station only.  The start station is where a trip begins and is, therefore, more significant than the destination.  Not to say that the destination is insignificant, but if recommendations make starting a trip more available or appealing, then ridership should increase.

Harry Bridge Plaza, San Francisco Caltrain, and Embarcadero at Sansome are the top three stations by duration.  It is interesting to note that San Francisco Caltrain has 62% more trips vs the other two top three stations--ranked by duration.

### 4.0 Trip Duration Distribution by Day of the Week

In [ ]:
%%bigquery week_day_dist_2016
SELECT *
FROM(
SELECT EXTRACT(year FROM start_date) AS date_year, 
       EXTRACT(dayofweek FROM start_date) AS date_day_num,
       FORMAT_DATETIME("%A", DATETIME(start_date)) AS date_day, 
       round(sum(duration_sec)/3600, 0) AS total_duration_hr,
       round(sum((((duration_sec)/60)/2692370)*100),2) as distribution
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY date_year, date_day_num, date_day
ORDER BY date_year, date_day_num
)
WHERE date_year = 2016

In [ ]:
%%bigquery week_day_dist_2015
SELECT *
FROM(
SELECT EXTRACT(year FROM start_date) AS date_year, 
       EXTRACT(dayofweek FROM start_date) AS date_day_num,
       FORMAT_DATETIME("%A", DATETIME(start_date)) AS date_day, 
       round(sum(duration_sec)/3600, 0) AS total_duration_hr,
       round(sum((((duration_sec)/60)/4578562)*100),2) as distribution
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY date_year, date_day_num, date_day
ORDER BY date_year, date_day_num
)
WHERE date_year = 2015

In [ ]:
%%bigquery week_day_dist_2014
SELECT *
FROM(
SELECT EXTRACT(year FROM start_date) AS date_year, 
       EXTRACT(dayofweek FROM start_date) AS date_day_num,
       FORMAT_DATETIME("%A", DATETIME(start_date)) AS date_day, 
       round(sum(duration_sec)/3600, 0) AS total_duration_hr,
       round(sum((((duration_sec)/60)/4673800)*100),2) as distribution
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
GROUP BY date_year, date_day_num, date_day
ORDER BY date_year, date_day_num
)
WHERE date_year = 2014

#### 4.1.1 - 2016 Trip Duration Distribution by Day of the Week

In [ ]:
week_day_dist_2016
week_day_dist_2016.set_index('date_day', inplace=True, drop=True)
week_day_dist_2016[['total_duration_hr','distribution']]

#### 4.1.2 - 2015 Trip Duration Distribution by Day of the Week

In [ ]:
week_day_dist_2015
week_day_dist_2015.set_index('date_day', inplace=True, drop=True)
week_day_dist_2015[['total_duration_hr','distribution']]

#### 4.1.3 - 2014 Trip Duration Distribution by Day of the Week

In [ ]:
week_day_dist_2014
week_day_dist_2014.set_index('date_day', inplace=True, drop=True)
week_day_dist_2014[['total_duration_hr','distribution']]

In [ ]:
fig, ax = plt.subplots(3,1,figsize = (16,26))
# width = 0.35
ax[0].bar(week_day_dist_2016.index, week_day_dist_2016.total_duration_hr,color='b')
ax[0].set_title('2016 Total Trip Duration by Day of the Week (Jan - Aug)',fontweight='bold',fontsize=24)
ax[0].set_xlabel('Day of the Week', fontsize=20, weight='bold')
ax[0].set_ylabel('Total Trip Duration (hr)', fontsize=20, weight='bold');
ax[0].set_xticklabels(week_day_dist_2016.index,fontsize=16); 
ax[0].set(ylim=(0,8000))
ax[0].set_axisbelow(True)
ax[0].grid(True, axis='y')


ax[1].bar(week_day_dist_2015.index, week_day_dist_2015.total_duration_hr,color='g')
ax[1].set_title('2015 Total Trip Duration by Day of the Week',fontweight='bold',fontsize=24)
ax[1].set_xlabel('Day of the Week', fontsize=20, weight='bold')
ax[1].set_ylabel('Total Trip Duration (hr)', fontsize=20, weight='bold');
ax[1].set_xticklabels(week_day_dist_2015.index,fontsize=16); 
ax[1].set(ylim=(0,14000))
ax[1].set_axisbelow(True)
ax[1].grid(True, axis='y')

ax[2].bar(week_day_dist_2014.index, week_day_dist_2014.total_duration_hr,color='r')
ax[2].set_title('2014 Total Trip Duration by Day of the Week',fontweight='bold',fontsize=24)
ax[2].set_xlabel('Day of the Week', fontsize=20, weight='bold')
ax[2].set_ylabel('Total Trip Duration (hr)', fontsize=20, weight='bold');
ax[2].set_xticklabels(week_day_dist_2014.index,fontsize=16); 
ax[2].set(ylim=(0,14000))
ax[2].set_axisbelow(True)
ax[2].grid(True, axis='y')

fig.tight_layout(pad=5)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (16,12))
ax.plot(week_day_dist_2015.index, week_day_dist_2015.total_duration_hr,color='g', linewidth=4, marker='o', markersize=8)
ax.plot(week_day_dist_2014.index, week_day_dist_2014.total_duration_hr,color='r', linewidth=4, marker='o', markersize=8)
ax.set_title('2015 vs. 2014 Total Trip Duration by Day of the Week',fontweight='bold',fontsize=24)
ax.set_xlabel('Day of the Week', fontsize=20, weight='bold')
ax.set_ylabel('Total Trip Duration (hr)', fontsize=20, weight='bold');
ax.set_xticklabels(week_day_dist_2015.index,fontsize=16); 
ax.set(ylim=(6000,14000))
ax.set_axisbelow(True)
ax.grid(True, axis='y')
ax.legend(prop={'size':14}, labels=['2015', '2014'], loc='best')

plt.show()

#### 4.1.4 Subscriber vs. Non-subscriber

In [ ]:
%%bigquery subscriber_vs_customer

SELECT date_year, subscriber_type, round(sum(duration_sec/3600),2) total_duration_hr
FROM(
SELECT *, EXTRACT(YEAR FROM start_date) date_year
FROM bigquery-public-data.san_francisco.bikeshare_trips 
WHERE duration_sec <= 28800)
GROUP BY 1,2
ORDER BY 1 DESC, 2 DESC

In [ ]:
subscriber_vs_customer

### 4.2 Summary

2015 vs. 2014 ridership was higher during the week but 2015 ridership decreased dramatically during the weekend when compared to 2014.  2014 had more consistent ridership on the weekends.  2014 logged a total of 77,896.97 trip hours.  While 2015 experienced a dip of 2% logging 76,309.37 trip hours.  From 2014 to 2015, subscription hours increased by 12.2%; however, non-subscription hours decreased by 18.3%.  Assuming non-subscription riders rode more during the weekend, then the significant decrease in 2015 weekend rides would be explained.

### 5.0 Trip Duration Distribution by Month

#### 5.1.1 - 2016 Trip Duration Distribution by Month

In [ ]:
%%bigquery monthly_trips_2016
SELECT *, round(total_trip_time_hr/num_days,2) avg_hour_per_day
FROM(
SELECT
extract(month FROM start_date) AS date_month_num,
FORMAT_DATETIME("%B", DATETIME(start_date)) AS date_month,
max(extract(day FROM start_date)) AS num_days,
count(DISTINCT trip_id) AS num_trips,
round(sum(duration_sec/3600), 2) AS total_trip_time_hr,
round(min(duration_sec/60),2) AS min_trip_time,
round(max(duration_sec/60),2) AS max_trip_time,
round(avg(duration_sec/60)) AS avg_trip_time_min,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
WHERE extract(year from start_date) = 2016
GROUP BY date_month_num, date_month
ORDER BY date_month_num
)

In [ ]:
monthly_trips_2016.set_index('date_month', inplace=True, drop=True)
monthly_trips_2016.drop(columns='date_month_num', inplace=True)

In [ ]:
monthly_trips_2016

In [ ]:
monthly_trips_2016

In [ ]:
round(monthly_trips_2016.total_trip_time_hr.sum(),2)

#### 5.1.2 - 2015 Trip Duration Distribution by Month

In [ ]:
%%bigquery monthly_trips_2015

SELECT *, round(total_trip_time_hr/num_days,2) avg_hour_per_day
FROM(
SELECT
extract(month FROM start_date) AS date_month_num,
FORMAT_DATETIME("%B", DATETIME(start_date)) AS date_month,
max(extract(day FROM start_date)) AS num_days,
count(DISTINCT trip_id) AS num_trips,
round(sum(duration_sec/3600), 2) AS total_trip_time_hr,
round(min(duration_sec/60),2) AS min_trip_time,
round(max(duration_sec/60),2) AS max_trip_time,
round(avg(duration_sec/60)) AS avg_trip_time_min,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
WHERE extract(year from start_date) = 2015
GROUP BY date_month_num, date_month
ORDER BY date_month_num
)

In [ ]:
monthly_trips_2015.set_index('date_month', inplace=True, drop=True)
monthly_trips_2015.drop(columns='date_month_num', inplace=True)

In [ ]:
monthly_trips_2015

In [ ]:
monthly_trips_2015.total_trip_time_hr[:8].sum()

In [ ]:
monthly_trips_2015.total_trip_time_hr.sum()

#### 5.1.3 - 2014 Trip Duration Distribution by Month

In [ ]:
%%bigquery monthly_trips_2014

SELECT *, round(total_trip_time_hr/num_days,2) avg_hour_per_day
FROM(
SELECT
extract(month FROM start_date) AS date_month_num,
FORMAT_DATETIME("%B", DATETIME(start_date)) AS date_month,
max(extract(day FROM start_date)) AS num_days,
count(DISTINCT trip_id) AS num_trips,
round(sum(duration_sec/3600), 2) AS total_trip_time_hr,
round(min(duration_sec/60),2) AS min_trip_time,
round(max(duration_sec/60),2) AS max_trip_time,
round(avg(duration_sec/60)) AS avg_trip_time_min,
FROM `studied-bounty-276502.project_01.trips_lesseq_8hr`
WHERE extract(year from start_date) = 2014
GROUP BY date_month_num, date_month
ORDER BY date_month_num
)

In [ ]:
monthly_trips_2014.set_index('date_month', inplace=True, drop=True)
monthly_trips_2014.drop(columns='date_month_num', inplace=True)

In [ ]:
monthly_trips_2014

In [ ]:
monthly_trips_2014.total_trip_time_hr[:8].sum()

In [ ]:
monthly_trips_2014.total_trip_time_hr.sum()

#### 5.1.4 - Summary
This data analysis was constrained by rides that were 8 hours or less.
* Total hours for 2016** = 44872.85     
* Total hours for 2015 = 76309.37
* Total hours for 2014 = 77896.67

Years constrained to Aug will serve as better comparison for 2016.
* 2016 Jan - Aug hours** = 44872.85
* 2015 Jan - Aug hours = 54271.54
* 2015 Jan - Aug hours = 53162.95

** Data available from Jan - Aug

### 5.2 Plot Total Trip Duration by Month

In [ ]:
fig, ax = plt.subplots(3,1,figsize = (16,26), sharex=True,)
ax[0].bar(monthly_trips_2016.index, monthly_trips_2016.total_trip_time_hr,color='b')
ax[0].set_title('2016 Total Trip Duration by Month (Jan - Aug)',fontweight='bold',fontsize=24)
ax[0].set_xlabel('Month', fontsize=20, weight='bold')
ax[0].set_ylabel('Total Trip Duration (hr)', fontsize=20, weight='bold');
ax[0].set_xticklabels(monthly_trips_2016.index,fontsize=16,rotation=45); 
ax[0].set(ylim=(0,10000))
ax[0].set_axisbelow(True)
ax[0].grid(True, axis='y')

ax[1].bar(monthly_trips_2015.index, monthly_trips_2015.total_trip_time_hr,color='g')
ax[1].set_title('2015 Total Trip Duration by Month',fontweight='bold',fontsize=24)
ax[1].set_xlabel('Month', fontsize=20, weight='bold')
ax[1].set_ylabel('Total Trip Duration (hr)', fontsize=20, weight='bold');
ax[1].set_xticklabels(monthly_trips_2015.index,fontsize=16,rotation=45); 
ax[1].set(ylim=(0,10000))
ax[1].set_axisbelow(True)
ax[1].grid(True, axis='y')

ax[2].bar(monthly_trips_2014.index, monthly_trips_2014.total_trip_time_hr,color='r')
ax[2].set_title('2014 Total Trip Duration by Month',fontweight='bold',fontsize=24)
ax[2].set_xlabel('Month', fontsize=20, weight='bold')
ax[2].set_ylabel('Total Trip Duration (hr)', fontsize=20, weight='bold');
ax[2].set_xticklabels(monthly_trips_2014.index,fontsize=16, rotation=45); 
ax[2].set(ylim=(0,10000))
ax[2].set_axisbelow(True)
ax[2].grid(True, axis='y')

# Restored xtick labels after sharex=True
for a in fig.axes:
    a.tick_params(
    axis='x',           # changes apply to the x-axis
    which='both',       # both major and minor ticks are affected
    bottom=True,
    top=False,
    labelbottom=True)

fig.tight_layout(pad=5)

plt.show()

### 5.3 Plot Avgerage Daily Total of Trip Duration by Month

In [ ]:
fig, ax = plt.subplots(3,1,figsize = (16,26), sharex=True,)
ax[0].bar(monthly_trips_2016.index, monthly_trips_2016.avg_hour_per_day,color='b')
ax[0].set_title('2016 Average Daily Total of Trip Duration by Month (Jan - Aug)',fontweight='bold',fontsize=24)
ax[0].set_xlabel('Month', fontsize=20, weight='bold')
ax[0].set_ylabel('Total Trip Duration (hr)', fontsize=20, weight='bold');
ax[0].set_xticklabels(monthly_trips_2016.index,fontsize=16,rotation=45); 
ax[0].set(ylim=(0,300))
ax[0].set_axisbelow(True)
ax[0].grid(True, axis='y')


ax[1].bar(monthly_trips_2015.index, monthly_trips_2015.avg_hour_per_day,color='g')
ax[1].set_title('2015 Average Daily Total of Trip Duration by Month',fontweight='bold',fontsize=24)
ax[1].set_xlabel('Month', fontsize=20, weight='bold')
ax[1].set_ylabel('Total Trip Duration (hr)', fontsize=20, weight='bold');
ax[1].set_xticklabels(monthly_trips_2015.index,fontsize=16,rotation=45); 
ax[1].set(ylim=(0,300))
ax[1].set_axisbelow(True)
ax[1].grid(True, axis='y')

ax[2].bar(monthly_trips_2014.index, monthly_trips_2014.avg_hour_per_day,color='r')
ax[2].set_title('2014 Average Daily Total of Trip Duration by Month',fontweight='bold',fontsize=24)
ax[2].set_xlabel('Month', fontsize=20, weight='bold')
ax[2].set_ylabel('Total Trip Duration (hr)', fontsize=20, weight='bold');
ax[2].set_xticklabels(monthly_trips_2014.index,fontsize=16, rotation=45); 
ax[2].set(ylim=(0,300))
ax[2].set_axisbelow(True)
ax[2].grid(True, axis='y')

# Restored xtick labels after sharex=True
for a in fig.axes:
    a.tick_params(
    axis='x',           # changes apply to the x-axis
    which='both',       # both major and minor ticks are affected
    bottom=True,
    top=False,
    labelbottom=True)

fig.tight_layout(pad=5)

plt.show()

### 5.4 Summary

The data shows that ridership decreased from 2014 to 2015.  With the data available from 2016 and projections made from 2014 and 2015, 2016 will most likely end up a down year as well.  From the above graphs, it is evident that the total duration of trips and average daily total of trips decreases during November and December. The cold/rainy weather of November and December could contribute to the downturn in ridership.  Motivating/incentivizing customers to ride during November and December would increase ridership during the slowest months of the year.

In this scenario, the average daily total of trip duration paints a better picture of the full story.  Some months have more days than other months.  Therefore comparing the total trip duration by month will represent an unfair distribution.

### 6.0 Station Capacity

#### 6.1 Station Capacity (from bikeshare_stations)

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
%%bigquery station_capacity
SELECT station_id, name, dockcount
FROM bigquery-public-data.san_francisco.bikeshare_stations
ORDER BY station_id

In [ ]:
station_capacity.head(75)

#### 6.2 Calculate Station Capacity From Most Recent Time Stamp (more accurate from bikeshare_status)

A bit of commentary on the code below.

It takes the most recent poll from the bikeshare_status table for each station, then adds the bike availability to the dock availability for an up-to-the-minute station capacity.  This code would detect if a dock were added or removed from a station.

In [ ]:
%%bigquery station_capacity_most_recent
SELECT * , (bikes_available + docks_available) as station_capacity
FROM(
SELECT t1.station_id, t1.bikes_available, t1.docks_available  FROM bigquery-public-data.san_francisco.bikeshare_status as t1
JOIN(
SELECT station_id, max(time) as mostrecent
FROM bigquery-public-data.san_francisco.bikeshare_status
GROUP BY station_id) as t2
ON t1.station_id = t2.station_id and t1.time=t2.mostrecent
ORDER BY t1.station_id)

In [ ]:
station_capacity_most_recent.set_index('station_id', inplace=True, drop=True)
station_capacity_most_recent.drop(columns=['bikes_available','docks_available'], inplace=True)

In [ ]:
station_capacity_most_recent.head(25)

### 6.3 Summary
The bike station capacity will be used in further analysis.

### 7.0 2016 Top Stations Low Bikes/Docks Percent During Busiest Hours 5AM to 8PM

#### Views used to create low_percent_by_top_stations_5to20_2016

##### View 1

Below is the bq I used to create the percentage of time that a particular bike station has low bikes available or low dock space available.  In order to calculate the "station low" percentage, I counted the total number of occurrences when bike availability was low (0,1, or 2 bikes available) and add that to the total of occurrences when dock availability was low.  I then divide that number by the total number of polls made on the bike station and multiply by 100 to derive the percentage of time that bikes/docks are low at a particular station.  

I define low as 0, 1, or 2.  Interestingly, I am using both the bike availability low and dock availability low to calculate this metric.  Here is why: If there are zero bikes available, no trips will be started.  Also, if 1 or 2 bikes are available, there is a possibility that the bikes remaining in the dock may be damaged. Therefore a customer will not check the bike out.  On the other hand, low docking stations may deter customers from starting a trip.  They may feel that if there are low amounts of docking stations at the current station, then there may not be any available docking stations at the next station.

I considered only the hours of 5 AM to 8 PM for this view.  The reason for doing so is because 96% of all trips are within that timeframe. Earlier in the analysis, I calculated this figure.  This will give better insight into the station bike/dock availability during the times that matter.  If hours outside of this time frame were considered, the average percent during peak hours would not be represented.  My intent was to capture the customers' perception of bike stations--which accomplished by analyzing the busiest hours of the day 5 AM - 8 PM.

In [ ]:
%%bigquery station_low_percent_5to20_2016 

SELECT station_id, round(AVG(low_percent),2) low_percent_avg
FROM(
SELECT *, round(((bikes_low + docks_low)/docks_count)*100,2) low_percent
FROM(
    SELECT EXTRACT(YEAR FROM time) AS date_year,
           EXTRACT(HOUR FROM time) date_hour,
           station_id,
           SUM(case when bikes_available IN (0,1,2) THEN 1 ELSE 0 END) as bikes_low,
           COUNT(bikes_available) as bikes_count,
           SUM(case when docks_available IN (0,1,2) THEN 1 ELSE 0 END) as docks_low,
           COUNT(docks_available) as docks_count
    FROM bigquery-public-data.san_francisco.bikeshare_status
    GROUP BY date_year, date_hour, station_id
    ORDER BY date_year DESC, station_id
    )
WHERE date_year = 2016 and date_hour BETWEEN 5 AND 20
ORDER BY station_id, date_hour)
GROUP BY station_id
ORDER BY station_id

In [ ]:
station_low_percent_5to20_2016.head()

##### View 2

Below is the bq I used to create a view with a few calculations regarding trips less than 8 hours.  The column that I sought to create was percent_trip_time.  This column represents the percent of total trips that the corresponding station contributes to the overall trip duration.  For example, in 2016, 7.43% of all trips started from Harry Bridges Plaza.  I will use the percent of total trips joined with the percent of bike station low percent, and all of that joined with the dock capacity to present a recommendation.  

In [ ]:
%%bigquery start_station_lesseq_8_2016

SELECT *
FROM(
    SELECT 
        extract(year from start_date) AS date_year,
        start_station_id,
        max(start_station_name) station_name,
        count(distinct trip_id) AS num_trips,
        round(sum(duration_sec)/60,0) AS total_trip_time,
        round(avg(duration_sec/60),0) AS avg_trip_time,
        round(sum((((duration_sec)/60)/2692370)*100),2) AS percent_trip_time,
        round(min(duration_sec/60),1) AS min_trip_time,
        round(max(duration_sec/60),1) AS max_trip_time
    FROM studied-bounty-276502.project_01.trips_lesseq_8hr
    GROUP BY 1,2
    ORDER BY 7 DESC
    )
WHERE date_year = 2016

In [ ]:
start_station_lesseq_8_2016.head(100)

##### View 3

Below is the bq I used to create a view with the zip codes, city, ad county for each station.

In [ ]:
%%bigquery station_zip_codes

WITH t1 AS 
(
SELECT ST_GEOGPOINT(longitude, latitude) point, station_id, station_name
FROM(
    SELECT station_id, max(name) AS station_name, max(latitude) as latitude, max(longitude) as longitude FROM `bigquery-public-data.san_francisco.bikeshare_stations`
    GROUP BY station_id
    ORDER BY station_id
    )
)
SELECT t1.station_id, t1.station_name, zip_code, city, county
FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
JOIN t1 ON ST_WITHIN(point, zip_code_geom)
ORDER BY station_id

In [ ]:
station_zip_codes.head()

#### Table 7.1
This table contains the percent of trips, by duration, that start from a particular station and the percent of time that station has either low bike availability or low dock availability.  I also included the dock count of each station.

In [ ]:
%%bigquery low_percent_by_top_stations_5to20_2016

SELECT t03.*, t04.dockcount dockcount FROM bigquery-public-data.san_francisco.bikeshare_stations as t04
JOIN(
SELECT 
    t01.start_station_id, 
    t01.start_station_name, 
    t01.percent_trip_time,
    t02.low_percent_avg, 
FROM studied-bounty-276502.project_01.start_station_lesseq_8_2016 AS t01
INNER JOIN studied-bounty-276502.project_01.station_low_percent_5to20_2016 AS t02 ON t01.start_station_id = t02.station_id
ORDER BY percent_trip_time DESC) as t03
ON t04.station_id = t03.start_station_id
ORDER BY percent_trip_time DESC

In [ ]:
low_percent_by_top_stations_5to20_2016.head(10)

#### Table 7.2
This table contains the percent of trips, by duration, that start from a particular station and the percent of time that station has either low bike availability or low dock availability.  I also included the dock count of each station.

In [ ]:
%%bigquery low_percent_zipcodes_by_top_stations_5to20_2016

SELECT 
    t01.start_station_id, 
    t01.start_station_name, 
    t01.percent_trip_time,
    t01.low_percent_avg, 
    t01.dockcount,
    t02.zip_code,
    t02.city,
    t02.county
FROM studied-bounty-276502.project_01.prejoin_duration_lowcapacity AS t01
JOIN studied-bounty-276502.project_01.prejoin_zip_codes AS t02 ON t01.start_station_id = t02.station_id
ORDER BY percent_trip_time DESC

In [ ]:
low_percent_zipcodes_by_top_stations_5to20_2016

#### Calculations

The top ten stations make up 46% of business.

In [ ]:
low_percent_by_top_stations_5to20_2016.percent_trip_time.head(10).sum()

The top ten stations average a low bike/dock availability 15% of the time.

In [ ]:
round(low_percent_by_top_stations_5to20_2016.low_percent_avg.head(10).mean(),2)

The top five stations comprise 30% of business.

In [ ]:
round(low_percent_by_top_stations_5to20_2016.percent_trip_time[:5].sum(),2)

The top five stations average a low bike/dock availability 21.2% of the time.

In [ ]:
round(low_percent_by_top_stations_5to20_2016.low_percent_avg[:5].mean(),2)

The top three stations account for 20% of business.

In [ ]:
round(low_percent_by_top_stations_5to20_2016.percent_trip_time[:3].sum(),2)

The top three stations average a low bike/dock availability 23.7% of the time.

In [ ]:
round(low_percent_by_top_stations_5to20_2016.low_percent_avg[:3].mean(),2)

### 7.1 Summary

This is my favorite analysis, partly because I learned how to join three tables and two views, and partly because the insights are fantastic.  From the table, it is evident that the top ten starting stations account for 46% of all 2016 trip duration.  When averaging low bike/rack availability across the top ten stations, it becomes apparent that these stations are low on bikes or racks 15% of the time.  As we look at the top three bike stations, which account for 21% of the business, that number jumps to 23.6%.  From the outside looking in, one way to generate more rides would be to decrease the low bike and rack availability by at least 10% initially. 

---
# What are the most popular commuter trips?

#### Commuter trip assumptions:
1. Commuter trips will initially be limited to 30 minutes or less--95% of trips fall within this time frame.  My definition of commute is repeated travel between locations.  I will filter by morning  hours 5 AM to 10 AM in an attempt to capture on leg of the trip--not the leg back.  The avg commuter trip duration will ultimately determine the window of time for a commuter trip.
2. Subscribers will be considered commuters.  However, customers who fall within the bounds of subscriber average trip times will not be left out of the analysis.  They may be commuters, but it is less likely that a customer would commute frequently without subscribing.
3. A commuter trip starts at one station and ends at another station.

### 1.0 Calculate the Average Commuter Trip Duration

The average commuter trip is 8min 41sec.  The standard deviation is 259 sec.  Subscribers make up 84% of trips 30 mins or less.  

I calculate the average commuter trip by taking the avg of all subscriber trips in the dataset.   I also calculated the standard deviation and the percent of total trips by subscriber vs. non-subscriber.  Remember, all of this calculated while examining trips of less than 30 minutes.

In [ ]:
%%bigquery bikeshare_trips_less_30_min

SELECT 
subscriber_type, 
ROUND(AVG(duration_sec),2) avg_dur_sec,
ROUND(STDDEV(duration_sec),0) stddev_sec,
ROUND(((SUM(duration_sec)/3600)/144878)*100,2) tot_dur_percent
FROM bigquery-public-data.san_francisco.bikeshare_trips 
WHERE duration_sec <=1800
GROUP BY subscriber_type
ORDER BY tot_dur_percent DESC

In [ ]:
bikeshare_trips_less_30_min

### 2.0 Determine Commuter Trip Time Frame

Subscribers make up 93.56% percent of trips between 262 and 780 seconds.  I determined this by summing a case function for subscribers among the dataset.  Also, I calculated that 67.37% of all trips less than 30 mins fall between 262 and 780 seconds.  Through my calculations and logical reasoning, I can assume that 64.37% (95% * 67.63% = 64.37%) of all trips are commuter trips.  This amount of commuter trips seems plausible.

In [ ]:
%%bigquery time_for_trip_percent_final

SELECT *, 
  round((subscriber_count/(subscriber_count + customer_count))*100,2) sub_percent, 
  round(((subscriber_count + customer_count)/936311)*100, 2) percent_of_all_trips_between_262_780
FROM(
SELECT 
  SUM(case when subscriber_type = 'Subscriber' then 1 else 0 end) subscriber_count,
  SUM(case when subscriber_type = 'Customer' then 1 else 0 end) customer_count
FROM bigquery-public-data.san_francisco.bikeshare_trips 
WHERE duration_sec BETWEEN 262 AND 780)

In [ ]:
time_for_trip_percent_final

### 3.0 Determine Most Popular Commuter Trips 

To determine the most popular commuter trips, I first extract the hour and day from each trip's timestamp.  Next, I constrain the return to the hours of 5 AM to 10 AM and weekdays (Mon, Tue, Wed, Thu, Fri).  The hours of 5 AM and 10 AM are reasonable time frames for commutes, as well as during the weekdays.  All of this using the time frame of 262 to 780 seconds for trip duration, and grouped by the start and end station, then ordered by the number of trips, returns the most popular commuter trips.

In [ ]:
%%bigquery pop_commuter_trips_morning

SELECT 
    start_station_name, 
    end_station_name, 
    COUNT(DISTINCT trip_id) num_trips,
FROM(
    SELECT
        EXTRACT(HOUR FROM start_date) date_hour,
        FORMAT_DATETIME("%a", DATETIME(start_date)) day_of_week,
        start_station_name, 
        end_station_name,
        trip_id
    FROM bigquery-public-data.san_francisco.bikeshare_trips 
    WHERE duration_sec BETWEEN 262 AND 780
    )
WHERE date_hour BETWEEN 5 AND 10 and day_of_week IN ('Mon', 'Tue', 'Wed', 'Thur', 'Fri') and start_station_name <> end_station_name
GROUP BY start_station_name, end_station_name
ORDER BY num_trips DESC

In [ ]:
%%bigquery pop_commuter_trips_evening

SELECT 
    start_station_name, 
    end_station_name, 
    COUNT(DISTINCT trip_id) num_trips,
FROM(
    SELECT
        EXTRACT(YEAR FROM start_date) date_year,
        EXTRACT(HOUR FROM start_date) date_hour,
        FORMAT_DATETIME("%a", DATETIME(start_date)) day_of_week,
        start_station_name, 
        end_station_name,
        trip_id
    FROM bigquery-public-data.san_francisco.bikeshare_trips 
    WHERE duration_sec BETWEEN 262 AND 780
    )
WHERE date_hour BETWEEN 15 AND 20 and day_of_week IN ('Mon', 'Tue', 'Wed', 'Thur', 'Fri')  and start_station_name <> end_station_name
GROUP BY start_station_name, end_station_name
ORDER BY num_trips DESC

In [ ]:
pop_commuter_trips_morning.head(20)

In [ ]:
pop_commuter_trips_morning.head(5)

In [ ]:
pop_commuter_trips_evening.head(5)

#### The top 5 most popular commuter trips are (drum roll)....

In [ ]:
pop_commuter_trips_morning.head(5)

It is somewhat reassuring to see that three of the most popular trips start station at the beginning of the day become the end stations at the end of the day.  Another observation is the top commuter trip (in the morning) becomes the top trip start station and end station reversed in the evening. Harry Bridges Plaza to 2nd at Townsend is the most popular trip in the morning while 2nd at Townsend to Harry Bridges Plaza is the most popular trip in the evening.